In [31]:
import csv
import psycopg2
import psycopg2.extras
import glob

In [32]:
def connectDB():
    conn = psycopg2.connect(host="localhost", dbname="postgres", user='postgres', port=5432, password='1111')
    return conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [33]:
def conDB():
    con = psycopg2.connect(host="localhost", dbname="postgres", user='postgres', port=5432, password='1111')
    return con

In [34]:
def getSchemaList():
        sql="""SELECT SCHEMA_NAME FROM information_schema.schemata
 WHERE SCHEMA_NAME NOT IN ('information_schema','public','pg_catalog','pg_toast_temp_1','pg_temp_1','pg_toast')"""
        conn = conDB()
        cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        cur.execute(sql)
        rows = cur.fetchall()
        cur.close()
        conn.close()
        return rows

In [35]:
def createTables():
        commands = ("DROP TABLE IF EXISTS Data_Dictionary",
                """
            CREATE TABLE Data_Dictionary(dd_Schema_Name VARCHAR,dd_Table_Name VARCHAR,dd_Column_Name VARCHAR,
dd_Data_Type VARCHAR,dd_Data_Length INTEGER,is_Nullable VARCHAR,dd_Default_Value VARCHAR,
is_PK VARCHAR,dd_Units VARCHAR,dd_Constraints VARCHAR,FK_Table VARCHAR,FK_Column VARCHAR,
is_unique VARCHAR,dd_Description Varchar)""" )
        conn = None
        try:
            conn = conDB()
            cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
            for command in commands:
                cur.execute(command)
            cur.close()
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close() 

In [36]:
def startEngine(SchemaName):
        conn = conDB()
        cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        table_names = getTableNames(SchemaName)
        content = []
        DescList=['primary key id','name of it','value of it','is active or not','1,2,3,4 priority level','user id','date when created','date when modified']
        for name in table_names:
                info_one_table = getInformationSchemasFromOneTable(name,SchemaName)
                for row_index, row in enumerate(info_one_table):
                        row_data = []
                        row_data.append(SchemaName)
                        row_data.append(name)
                        for item in row:
                                row_data.append(item)

                        if isPK(name, row[0],SchemaName) == True: 
                                row_data.append('Y')
                        else: row_data.append('')
                        row_data.append('')
                        row_data.append('')
                        fkdata = getFKs(name, row[0],SchemaName)
                        if fkdata!="":
                             fk_table,fk_col=fkdata.split()
                             row_data.append(fk_table)
                             row_data.append(fk_col)
                        else:
                             row_data.append('')
                             row_data.append('')

                        if isUnique(name, row[0],SchemaName) == True: 
                                row_data.append('Y')
                        else: row_data.append('')
                        row_data.append(DescList[row_index])
                        print(row_data)
                        print(row_index)
                        content.append(tuple(row_data))
                sql="""INSERT INTO data_dictionary (dd_Schema_Name,dd_Table_Name,dd_Column_Name,dd_Data_Type,dd_Data_Length,is_Nullable,dd_Default_Value,
is_PK,dd_Units,dd_Constraints,FK_Table,FK_Column,is_unique,dd_Description)
                VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
                cur.executemany(sql,content)
                conn.commit()
                content = []
          
        cur.close()        

In [37]:
def getInformationSchemasFromOneTable(table_name,SchemaName):
    cur = connectDB()
    cur.execute("SELECT column_name, data_type, character_maximum_length, is_nullable, column_default FROM INFORMATION_SCHEMA.COLUMNS \
                    WHERE table_name = '" + table_name + "' AND table_schema='"+SchemaName+"';")
    rows = cur.fetchall()
    return rows

In [38]:
def getTableNames(SchemaName):
    cur = connectDB()
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='"+SchemaName+"' AND table_type='BASE TABLE' ORDER BY table_name")
    rows = cur.fetchall()

    ret = []
    for row in rows:
        ret.append(row[0])
    
    return ret

In [39]:
def getFKs(table_name, column_name,SchemaName):
    cur = connectDB()
    cur.execute("SELECT kcu.column_name, ccu.table_name AS foreign_table_name, ccu.column_name AS foreign_column_name \
                    FROM information_schema.table_constraints AS tc JOIN information_schema.key_column_usage AS kcu ON tc.constraint_name = kcu.constraint_name \
                    JOIN information_schema.constraint_column_usage AS ccu ON ccu.constraint_name = tc.constraint_name \
                    WHERE tc.constraint_type = 'FOREIGN KEY' AND tc.table_name='" + table_name + "' AND tc.table_schema='"+SchemaName+"';")
    rows = cur.fetchall()
    ret = ""
    for row in rows:
        if row[0] == column_name:
            ret = row[1] + " " + row[2]
    return ret

In [40]:
def isPK(table_name, column_name,SchemaName):
    cur = connectDB()
    cur.execute("SELECT a.attname, format_type(a.atttypid, a.atttypmod) AS data_type FROM  pg_index i \
                    JOIN  pg_attribute a ON a.attrelid = i.indrelid \
                    AND a.attnum = ANY(i.indkey) WHERE  i.indrelid = '"+SchemaName+"." + table_name + "'::regclass AND i.indisprimary;")
    rows = cur.fetchall()
    for row in rows:
        if row[0] == column_name:
            return True
    return False

In [41]:
def isUnique(table_name, column_name,SchemaName):
    cur = connectDB()
    cur.execute("SELECT column_name FROM information_schema.table_constraints tc right \
                    JOIN information_schema.constraint_column_usage ccu \
                    ON tc.constraint_catalog = ccu.constraint_catalog \
                    AND tc.constraint_schema = ccu.constraint_schema \
                    AND tc.constraint_name = ccu.constraint_name and ccu.table_name in ('" + table_name + "') WHERE lower(tc.constraint_type) in ('unique')\
                    AND tc.table_schema='"+SchemaName+"';")
    rows = cur.fetchall()
    for row in rows:
        if row[0] == column_name:
            return True
    return False

In [42]:
createTables()

In [43]:
schemas = getSchemaList()
for s in schemas:
     startEngine(s[0])

['autho', 'user', 'user_id', 'integer', None, 'NO', "nextval('autho.user_user_id_seq'::regclass)", 'Y', '', '', '', '', '', 'primary key id']
0
['lookup', 'classification', 'lu_id', 'integer', None, 'NO', None, 'Y', '', '', '', '', '', 'primary key id']
0
['lookup', 'classification', 'lu_name', 'character varying', 50, 'NO', None, '', '', '', '', '', '', 'name of it']
1
['lookup', 'classification', 'lu_value', 'character varying', 50, 'NO', None, '', '', '', '', '', 'Y', 'value of it']
2
['lookup', 'classification', 'is_active', 'boolean', None, 'NO', None, '', '', '', '', '', '', 'is active or not']
3
['lookup', 'classification', 'default_priority', 'smallint', None, 'NO', None, '', '', '', '', '', '', '1,2,3,4 priority level']
4
['lookup', 'classification', 'create_user_id', 'integer', None, 'NO', None, '', '', '', 'user', 'user_id', '', 'user id']
5
['lookup', 'classification', 'create_date', 'timestamp without time zone', None, 'NO', "'2019-05-11 21:45:10.515089'::timestamp without